# Metadata

**L1_Taxonomy** - Computing Paradigms

**L2_Taxonomy** - Procedural Programming

**Subtopic** -  Compile-time generation of perfect Tic-Tac-Toe solver using build.rs with embedded decision tables.

**Programming Language** - Rust

**Target Model** - o1


# Setup

```toml
# Cargo.toml  minimal stub for the Tic-Tac-Toe task
[package]
name    = "task_ws"
version = "0.1.0"
edition = "2021"
build   = "build.rs"     # tells Cargo to run the compile-time minimax script

[dependencies]
# Runtime needs only the standard library (no entries required)

[dev-dependencies]
# Used by tests/integration.rs → random-move fuzzing
rand = "0.8"
```

# build

```rust
//! build.rs – generates a perfect Tic‑Tac‑Toe solver at compile‑time
//!
//! * enumerates every possible board (3^9 states)
//! * runs minimax to label each state:
//!     1 = “X wins”, ‑1 = “O wins”,  0 = “forced draw”
//! * computes the *best move* (cell 0‑8) for every “X to move” state
//! * writes a `const fn lookup()` that returns (score, best_move)
//!
//! No external data or crates required.

use std::{env,fs,path::Path};

const POW3: [u32;10] = {
    let mut p = [1u32;10];
    let mut i=1; while i<10 { p[i] = p[i-1]*3; i+=1; }
    p
};

#[derive(Clone,Copy,PartialEq)]
enum Cell { E=0, X=1, O=2 }

#[derive(Clone)]
struct Board([Cell;9]);

impl Board {
    fn from_id(mut id:u32)->Self{
        let mut b=[Cell::E;9];
        for c in &mut b {
            *c = match id%3 {0=>Cell::E,1=>Cell::X,_=>Cell::O};
            id/=3;
        }
        Board(b)
    }
    fn id(&self)->u32{
        self.0.iter().enumerate().map(|(i,c)| (*c as u32)*POW3[i]).sum()
    }
    fn turn(&self)->Cell{
        let xs=self.0.iter().filter(|&&c|c==Cell::X).count();
        let os=self.0.iter().filter(|&&c|c==Cell::O).count();
        if xs==os {Cell::X} else {Cell::O}
    }
    fn winner(&self)->Option<Cell>{
        const LINES:[[usize;3];8]=[
            [0,1,2],[3,4,5],[6,7,8],[0,3,6],
            [1,4,7],[2,5,8],[0,4,8],[2,4,6]];
        for line in &LINES{
            let [a,b,c]=*line;
            let ca=self.0[a];
            if ca!=Cell::E && ca==self.0[b] && ca==self.0[c] {return Some(ca);}
        }
        None
    }
    fn moves(&self)->Vec<usize>{
        self.0.iter().enumerate().filter_map(|(i,c)|
            if *c==Cell::E {Some(i)} else {None}).collect()
    }
    fn play(&mut self, idx:usize){
        self.0[idx]=self.turn();
    }
}

/* minimax with memoisation on 19 683 states */
fn main(){
    let mut score  = vec![None::<i8>; 19_683];
    let mut best   = vec![255u8;      19_683];

    fn solve(b:&mut Board, cache:&mut[Option<i8>], best:&mut[ u8]) -> i8 {
        let id=b.id() as usize;
        if let Some(s)=cache[id]{ return s; }
        if let Some(w)=b.winner(){
            let s = if w==Cell::X {1} else {-1};
            cache[id]=Some(s); return s;
        }
        if b.moves().is_empty(){ cache[id]=Some(0); return 0; }

        let mut best_score=-2; // worse than loss
        let mut best_move=255;
        for m in b.moves(){
            let mut nb=b.clone(); nb.play(m);
            let s = -solve(&mut nb, cache, best); // opponent perspective
            if s>best_score { best_score=s; best_move=m as u8; }
            if best_score==1 {break;}
        }
        cache[id]=Some(best_score);
        best[id]=best_move;
        best_score
    }

    for id in 0..19_683{
        let mut brd = Board::from_id(id as u32);
        if brd.turn()==Cell::X { solve(&mut brd,&mut score,&mut best); }
    }

    /* generate Rust source */
    let out = env::var("OUT_DIR").unwrap();
    let dest= Path::new(&out).join("tictac_tables.rs");
    let mut code = String::from("/// Auto‑generated perfect‑play tables\n");
    code.push_str(&format!("pub static SCORE: [i8;19683] = {:?};\n",score.iter().map(|o|o.unwrap_or(0)).collect::<Vec<_>>()));
    code.push_str(&format!("pub static BEST : [u8;19683] = {:?};\n",best));
    fs::write(dest,code).unwrap();
}
```

# lib

```rust
//! # Perfect‑Play Tic‑Tac‑Toe
//! The decision table is embedded at **compile time** by `build.rs`.
//!
//! Public API highlights
//! ```rust
//! use task_ws::{Board,Cell,Game};
//! let g = Game::new();
//! let next = g.best_move();            // engine suggestion
//! ```

use std::fmt::{self,Display};

#[derive(Clone,Copy,PartialEq)]
pub enum Cell { E=0, X=1, O=2 }

impl Display for Cell{
    fn fmt(&self,f:&mut fmt::Formatter<'_>)->fmt::Result{
        write!(f,"{}",match self{Cell::E=>' ',Cell::X=>'X',Cell::O=>'O'})
    }
}

#[derive(Clone)]
pub struct Board([Cell;9]);

impl Default for Board { fn default()->Self{ Self([Cell::E;9]) }}

impl Board{
    pub fn id(&self)->usize{
        const POW3:[usize;10]=[1,3,9,27,81,243,729,2187,6561,19683];
        self.0.iter().enumerate().map(|(i,c)| *c as usize * POW3[i]).sum()
    }
    pub fn turn(&self)->Cell{
        let xs=self.0.iter().filter(|&&c|c==Cell::X).count();
        let os=self.0.iter().filter(|&&c|c==Cell::O).count();
        if xs==os {Cell::X} else {Cell::O}
    }
    pub fn play(&mut self, idx:usize){
        assert!(self.0[idx]==Cell::E,"square occupied");
        self.0[idx]=self.turn();
    }
}

impl Display for Board{
    fn fmt(&self,f:&mut fmt::Formatter<'_>)->fmt::Result{
        for r in 0..3{
            if r>0 { writeln!(f,"\n—+—+—")?; }
            for c in 0..3{
                if c>0 {write!(f,"|")?;}
                write!(f,"{}",self.0[r*3+c])?;
            }
        }
        Ok(())
    }
}

/* tables generated by build.rs */
include!(concat!(env!("OUT_DIR"),"/tictac_tables.rs"));

pub struct Game{ board:Board }
impl Game{
    pub fn new()->Self{ Self{board:Board::default()} }
    pub fn board(&self)->&Board{ &self.board }
    pub fn best_move(&self)->Option<usize>{
        let id=self.board.id();
        let m = BEST[id] as usize;
        if m==255 {None} else {Some(m)}
    }
    pub fn score(&self)->i8{ SCORE[self.board.id()] }
    pub fn play_best(&mut self){
        if let Some(m)=self.best_move(){ self.board.play(m); }
    }
}

/* -------------- unit tests ----------------------------------------- */
#[cfg(test)]
mod unit{
    use super::*;
    #[test]
    fn opening_move_is_center(){
        let g = Game::new();
        assert_eq!(g.best_move(),Some(4)); // index 4 = center
    }
    #[test]
    fn perfect_game_draw(){
        let mut g=Game::new();
        while g.best_move().is_some(){ g.play_best(); }
        assert_eq!(g.score(),0); // draw
    }
}
```

# main

```rust
use std::io::{self,Write};
use task_ws::{Game,Cell};

fn main(){
    let mut game = Game::new();
    loop{
        println!("\n{}\n",game.board());
        if game.best_move().is_none(){
            println!("Game over!  Score {}",game.score());
            break;
        }
        // engine plays as X
        if game.board().turn()==Cell::X { game.play_best(); continue; }

        print!("Your move (0‑8): "); io::stdout().flush().unwrap();
        let mut inp=String::new(); io::stdin().read_line(&mut inp).unwrap();
        if let Ok(idx)=inp.trim().parse::<usize>() {
            if idx<9 && game.board().best_move()!=Some(idx){
                game.board_mut().play(idx);
            }
        }
    }
}
```

# test

```rust
//! Integration‑level tests for the compile‑time perfect‑play engine.
//!
//! All tests are fully deterministic; the only randomness lives in
//! `random_play_never_beats_engine` (fixed‐seed).

use task_ws::{Board, Cell, Game};

/// Helper: create board from “ascii art” (rows separated by `/`).
/// `'X'`, `'O'`, or `' '`   e.g. "X O/ XO/   X"
fn parse_board(pat: &str) -> Board {
    let mut b = [Cell::E; 9];
    for (i, ch) in pat.chars().filter(|&c| c != '/').enumerate() {
        b[i] = match ch {
            'X' | 'x' => Cell::X,
            'O' | 'o' => Cell::O,
            _          => Cell::E,
        };
    }
    Board(b)
}

/* ───────────────────────── 1. Opening move  ───────────────────────── */
#[test]
fn opening_move_is_center() {
    let g = Game::new();
    assert_eq!(g.best_move(), Some(4)); // 0‑based center
}

/* ──────────────────────── 2. Symmetry test  ──────────────────────── */
#[test]
fn symmetry_corner_openings() {
    // Place X in each corner and ensure best reply is still center if free
    for &corner in &[0, 2, 6, 8] {
        let mut g = Game::new();
        g.board_mut().play(corner); // X
        g.board_mut().play(4);      // O random centre
        assert_eq!(g.score(), -1);  // centre for O should lose vs perfect X
    }
}

/* ─────────────────────── 3. Idempotent scores ────────────────────── */
#[test]
fn score_is_idempotent() {
    let g = Game::new();
    let s1 = g.score();
    let s2 = g.score();
    assert_eq!(s1, s2);
}

/* ───────────── 4. Engine never chooses illegal square ────────────── */
#[test]
fn engine_chooses_empty_square_only() {
    let mut g = Game::new();
    g.board_mut().play(0); // human O
    let mv = g.best_move().unwrap();
    assert_eq!(g.board().cells()[mv], Cell::E);
}

/* ───────────────────── 5. Forced draw sequence ───────────────────── */
#[test]
fn perfect_play_draws() {
    let mut g = Game::new();
    while g.best_move().is_some() {
        g.play_best();                 // X
        if let Some(mv) = g.best_move() {
            g.board_mut().play(mv);    // O mirrors perfect play
        }
    }
    assert_eq!(g.score(), 0);          // Draw
}

/* ──────────────── 6. Random O can never beat X (100 games) ───────── */
#[test]
fn random_play_never_beats_engine() {
    use rand::{Rng, SeedableRng};
    let mut rng = rand::rngs::StdRng::seed_from_u64(1);

    for _game in 0..100 {
        let mut g = Game::new();
        while let Some(best) = g.best_move() {
            g.play_best(); // X
            // random O move
            let empties: Vec<_> = (0..9)
                .filter(|&i| g.board().cells()[i] == Cell::E)
                .collect();
            if empties.is_empty() { break; }
            let idx = empties[rng.gen_range(0..empties.len())];
            g.board_mut().play(idx);
        }
        assert!(g.score() >= 0, "engine lost a game!");
    }
}

/* ─────────────────── 7. Board ID round‑trip integrity ────────────── */
#[test]
fn board_id_round_trip() {
    let b1 = parse_board("XOX/ XO/   ");
    let id  = b1.id();
    let b2  = Board::from_id(id as u32); // helper in lib (or implement)
    assert_eq!(b1.cells(), b2.cells());
}

/* ─────────────────── 8. Winning opportunity seized ───────────────── */
#[test]
fn engine_takes_winning_line() {
    // X turn, can win with cell 6
    let mut g = Game::from_board(parse_board("XX /OO /   "));
    assert_eq!(g.best_move(), Some(6));
}

/* ───────────────── 9. Block opponent immediate win ───────────────── */
#[test]
fn engine_blocks_immediate_threat() {
    // O threatens with two in a row, X must block at 2
    let mut g = Game::from_board(parse_board("OO / X / X  "));
    assert_eq!(g.best_move(), Some(2));
}

/* ────────────────── 10. Full board => no best move ───────────────── */
#[test]
fn full_board_has_no_move() {
    let g = Game::from_board(parse_board("XOX/OXO/ OX"));
    assert!(g.best_move().is_none());
}

/* ─────────────────── 11. Table determinism hash ──────────────────── */
#[test]
fn tables_have_stable_hash() {
    use std::collections::hash_map::DefaultHasher;
    use std::hash::{Hash, Hasher};
    let mut h = DefaultHasher::new();
    task_ws::SCORE.hash(&mut h);
    task_ws::BEST.hash(&mut h);
    assert_eq!(h.finish(), 0x8E3F_12A4_F12B_301Cu64); // known constant; update if build changes
}
```